# Lab 12 Character Sequence RNN

In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(32147354)

In [2]:
#tf.set_random_seed(32147354)
sample = " 안녕하세요 이봉구입니다.반갑습니다"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

In [3]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell, if you want yo
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello, f you want you

In [4]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [5]:
X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [6]:
x_one_hot = tf.one_hot(X, num_classes)  
# one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
# num_classes 몇개의 원핫을 ? 유일한 문자열의 수만큼
# Shape이 어덯게 변하는지 살펴봐야 한다

In [7]:
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)

In [8]:
initial_state = cell.zero_state(batch_size, tf.float32)

In [9]:
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [10]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, 
                                            num_classes, 
                                            activation_fn=None)

In [11]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs,
                     [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)


In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(30):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))


0 loss: 2.76171 Prediction: 다다다세요이다다다다다다다다다다니다
1 loss: 2.57844 Prediction: 니니니니니니니니니다니다니다니니니다
2 loss: 2.42019 Prediction: 니니하세요 이봉니니니다다다니니니다
3 loss: 2.07159 Prediction: 니세세세요 이봉구니니다니다니니니니
4 loss: 1.72501 Prediction: 하세세세요 이봉구입니다다반갑습니다
5 loss: 1.40881 Prediction: 하세세세요 이봉구입니다.반갑습니다
6 loss: 1.11488 Prediction: 하하세세요 이봉구입니다.반갑습니다
7 loss: 0.827423 Prediction: 하하하세요 이봉구입니다.반갑습니다
8 loss: 0.595422 Prediction: 안녕하세요 이봉구입니다.반갑습니다
9 loss: 0.409593 Prediction: 안녕하세요 이봉구입니다.반갑습니다
10 loss: 0.27809 Prediction: 안녕하세요 이봉구입니다.반갑습니다
11 loss: 0.192144 Prediction: 안녕하세요 이봉구입니다.반갑습니다
12 loss: 0.128634 Prediction: 안녕하세요 이봉구입니다.반갑습니다
13 loss: 0.0857616 Prediction: 안녕하세요 이봉구입니다.반갑습니다
14 loss: 0.0565654 Prediction: 안녕하세요 이봉구입니다.반갑습니다
15 loss: 0.0367447 Prediction: 안녕하세요 이봉구입니다.반갑습니다
16 loss: 0.0239734 Prediction: 안녕하세요 이봉구입니다.반갑습니다
17 loss: 0.0161007 Prediction: 안녕하세요 이봉구입니다.반갑습니다
18 loss: 0.0112412 Prediction: 안녕하세요 이봉구입니다.반갑습니다
19 loss: 0.00810223 Prediction: 안녕하세요 이봉구입니다.반갑습니다
20 loss: 0.00594984 Prediction